In [2]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

num_classes = 4

images = []
labels = []

class_folders = ["glioma_tumor", "meningioma_tumor", "pituitary_tumor", "no_tumor"]

for class_index, class_folder in enumerate(class_folders):
    class_path = os.path.join("dataset", class_folder)
    class_files = os.listdir(class_path)
    
    for image_file in class_files:
        img = cv2.imread(os.path.join(class_path, image_file))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (64, 64))
        images.append(img)
        labels.append(class_index)

images = np.array(images)
labels = np.array(labels)

labels = tf.keras.utils.to_categorical(labels, num_classes)

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.3)
x_train = x_train.reshape(-1, 64, 64, 1)
x_test = x_test.reshape(-1, 64, 64, 1)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) 

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [3]:
model.fit(x_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
132/132 [==============================] - 11s 75ms/step - loss: 1.4203 - accuracy: 0.5681
Epoch 2/20
132/132 [==============================] - 10s 75ms/step - loss: 0.6073 - accuracy: 0.7429
Epoch 3/20
132/132 [==============================] - 10s 75ms/step - loss: 0.4423 - accuracy: 0.8119
Epoch 4/20
132/132 [==============================] - 10s 75ms/step - loss: 0.3604 - accuracy: 0.8579
Epoch 5/20
132/132 [==============================] - 10s 75ms/step - loss: 0.2616 - accuracy: 0.9021
Epoch 6/20
132/132 [==============================] - 10s 76ms/step - loss: 0.1894 - accuracy: 0.9319
Epoch 7/20
132/132 [==============================] - 10s 75ms/step - loss: 0.1484 - accuracy: 0.9455
Epoch 8/20
132/132 [==============================] - 10s 76ms/step - loss: 0.1193 - accuracy: 0.9600
Epoch 9/20
132/132 [==============================] - 10s 76ms/step - loss: 0.1113 - accuracy: 0.9612
Epoch 10/20
132/132 [==============================] - 10s 75ms/step - loss: 0.071

In [4]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

predictions = model.predict(x_test)

y_test_classes = np.argmax(y_test, axis=1)
predicted_classes = np.argmax(predictions, axis=1)

print("Classification Report:")
print(classification_report(y_test_classes, predicted_classes))

print("Confusion Matrix:")
print(confusion_matrix(y_test_classes, predicted_classes))

Test Loss: 0.3603
Test Accuracy: 0.9339
57/57 [==============================] - 1s 23ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.89      0.91       435
           1       0.90      0.86      0.88       453
           2       0.92      0.99      0.95       465
           3       0.99      0.99      0.99       447

    accuracy                           0.93      1800
   macro avg       0.93      0.93      0.93      1800
weighted avg       0.93      0.93      0.93      1800

Confusion Matrix:
[[388  39   8   0]
 [ 25 390  32   6]
 [  2   2 461   0]
 [  3   1   1 442]]


In [5]:
model.save('brain_tumor_model.h5')

c:\Users\Chandu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
